In [1]:
# Imports the function for calculating Cohen's kappa score 
from sklearn.metrics import cohen_kappa_score

In [50]:
import pandas as pd
import numpy as np

import re
import csv

Function for processing the file created by prompting Llama3 through the console:

In [3]:
def text_processing(filename):
    ''' 
    Opens the files created by prompting Llama3 through Powershell, and finds all instances of brackets, [],
    and adds these to a list.
    Then runs through each element in the list, which is now a string of numbers, and splits by comma
    and removes whitespace.
    `filename`: a textfile with output from prompting Llama3
    '''
    with open(filename, 'r', encoding='utf-16le') as infile:
        text = infile.read()
    
    bracket_list = re.findall(r'\[(.*?)\]', text)

    split_num_values = np.zeros((len(bracket_list),11))
    for i in range(len(bracket_list)):
        comma_sp = bracket_list[i].split(',')       # splits by comma
        x = np.zeros(11)
        for j in range(len(comma_sp)):
            x[j] = comma_sp[j].strip(' ')
        split_num_values[i] = x

    return split_num_values

### Creating instances for each file

Reading the pickle-file previosly created in as a dataframe

In [4]:
df = pd.read_pickle('df.pkl')

Converting this dataframe to an array to compare with the other values

In [5]:
ground_truth = df.iloc[0:, 2:-1].to_numpy()

In [6]:
run_1 = text_processing(r'C:\Users\maria\OneDrive - Universitetet i Oslo\VÅR2024\Textembedding\Drexl\llama_output\output_drexl_1.txt')
run_2 = text_processing(r'C:\Users\maria\OneDrive - Universitetet i Oslo\VÅR2024\Textembedding\Drexl\llama_output\output_drexl_2.txt')
run_3 = text_processing(r'C:\Users\maria\OneDrive - Universitetet i Oslo\VÅR2024\Textembedding\Drexl\llama_output\output_drexl_3.txt')

### Cohen's Kappa score for Llama3

Since the data from the Drexl-set can be categorized as more than one category we get a nested array containing the data. The function cohen_kappa_score cannot compute the score of nested arrays. The function below computes this score two different ways, first by line, compares each response to the ground truth, and then by category.

In [7]:
def nested_cohen(y1, y2):
    ''' 
    Computes Cohen's Kappa Score for a nested array by row and by column(category).
    `y1`: nested array
    `y2`: nested array
    '''
    # by row
    cohen_by_row = np.zeros(len(y1))
    for i in range(len(y1)):
        cohen_by_row[i] = cohen_kappa_score(y1[i],y2[i])

    # by column/category
    cohen_by_column = np.zeros(len(y1[0]))
    for i in range(len(y1[0])):
        y1_col = y1[:,i]
        y2_col = y2[:,i]
        cohen_by_column[i] = cohen_kappa_score(y1_col,y2_col)
        
    return cohen_by_row, cohen_by_column

We create a function to turn the output from the previous function into a dictionary containing values such as the mean, max and min

In [56]:
def data_to_dict(y1, y2):
    a, b = nested_cohen(y1, y2)
    datadict = {'Mean by line': np.mean(a), 
                'Max': np.max(a), 
                'Min': np.min(a), 
                'Standard deviation': np.std(a), 
                'First quantile': np.percentile(a, 25),
                'Median': np.percentile(a, 50),
                'Third quantile': np.percentile(a, 75),
                'Cohen\'s Kappa by category': b, 
                'Mean by category': np.mean(b)
                }
    
    return datadict

We run this function for each comparison we wish to make:

In [57]:
run_list = [ground_truth, run_1, run_2, run_3]
run_name = ['Ground truth', 'Run 1', 'Run 2', 'Run 3']

dict_list = []
for i in range(0,len(run_list)):
    for j in range(i+1, len(run_list)):
        d = data_to_dict(run_list[i], run_list[j])

        updict = {'Compared runs': f'{run_name[i]} and {run_name[j]}'}          # description of which runs are being compared    
        updict.update(d)                                                        # adds the dictionary as the first item
        dict_list.append(updict)

We then want to write this lilst of dictionaries to a csv-file:

In [58]:
# field names
fields = ['Compared runs', 'Mean by line', 'Max', 'Min', 'Standard deviation', 'First quantile',
          'Median', 'Third quantile', 'Cohen\'s Kappa by category', 'Mean by category']

filename = "IRR_results.csv"

with open(filename, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)         # creating a csv dict writer object
    
    writer.writeheader()                                        # writing headers (field names)
    writer.writerows(dict_list)                                 # writing data rows

### Cohen's Kappa Score for Centroid Method

In [14]:
print(f'{chr(954)}: Ground truth and Centroid Method = {cohen_kappa_score(ground_truth, embed_run)}')

NameError: name 'embed_run' is not defined

#### Excluded list

In [ ]:
ground_t_c = ground_truth.copy()
embed_ex = embed_converter('embed_news_excluded.csv')

Removes the "unknown" values, ie. the values where the greatest value in the distribution from the centroid method was less than 0.50, from both the ground truth and the list from the embeddings to get comparable data:

In [ ]:
i = 0
while i < len(embed_ex):
    if embed_ex[i] == 'unknown':
        embed_ex.pop(i)
        ground_t_c.pop(i)
    i += 1

In [ ]:
cohen_kappa_score(ground_t_c, embed_ex)